# Federal University of Pampa <www.unipampa.edu.br>
# Course: Deep Learning
# Author: Sandro Camargo <sandrocamargo@unipampa.edu.br>
# Single Neuron Logistic Regression Example
# Dataset: https://archive.ics.uci.edu/dataset/547/algerian+forest+fires+dataset

To open this code in your Google Colab environment, [click here](https://colab.research.google.com/github/Sandrocamargo/deep-learning/blob/master/dl_class03_LogisticRegressionSingleNeuron.ipynb).

A Python library is a collection of related functions. A library contains bundles of encapsuated code which can be used repeatedly in different programs.

In [ ]:
# Import Libraries
import keras # Neural Network Library
from keras import layers # Layers to a neural network
from keras import optimizers # optimizers
from keras.utils import plot_model # Print the network
import pandas as pd # Data Manipulation library
import numpy as np # Fast Numeric Computing library
import tensorflow as tf # Optimizers
import matplotlib.pyplot as plt # Plot library
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Loading dataset
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv', header=1, skiprows=[124,125,126,170])
# About the parameters
# Header=1: column names (day, month, year, ...) are in the line 1 of this CSV file
# skiprows=[124,125,126,170]: this lines, which not contains valid data, are not imported. If this parameter is missing, all lines are imported.

# inspecting columns and data types from "data" dataframe
data.info()

In [ ]:
# Store target column in y
# Store the inputs in X
y = data[data.columns[13]]
X = data.drop(columns=data.columns[13])

In [ ]:
# There are whitespaces in target column, is some samples.
print(y.value_counts())
y = pd.Series(y)
y = y.str.strip() # Remove whitespaces from extremes
print(y.value_counts())

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25,  random_state=1, stratify=y)

The dataset must be randomly splitted in two parts: training set and testing set. The main approaches to split are holdout and n-fold cross validation.
*   Training set is used for building (training) the model.
*   Testing set is used for testing the generalization ability of the model built.

Moreover, inputs($x$) and outputs($y$) must be splitted in each set.




In [ ]:
# Verifying dataset dimensions
print('The training dataset (inputs) dimensions are: ', train_x.shape)
print('The training dataset (outputs) dimensions are: ', train_y.shape)
print('The testing dataset (inputs) dimensions are: ', test_x.shape)
print('The testing dataset (outputs) dimensions are: ', test_y.shape)

The output is the column Classes. Its content is 'fire' or 'not fire'. But neural networks just deal with numbers. So, classes must be transformed in a binary column containing 0 for 'not fire' and 1 for 'fire'.

In [ ]:
# Mapping values to binary: hot encoding
binary_mapping = {'fire': 1, 'not fire': 0}

train_y_bin = train_y.map(binary_mapping)
test_y_bin = test_y.map(binary_mapping)

After creating the datasets, the next step is defining the architecture of our model.

It must be defined:


*   Architecture: in terms of neurons and layers
*   Optimizer: is the algorithm or method used to change the weights in order to minimize the loss function.

The last step is compiling the model. In this step the loss function, the optimizer and the evaluation metrics must be defined.

In [ ]:
# Function to define model architecture
def build_model():
  # Defining the architecture
  # Sequential = Feedforward Neural Network
  # 1 single neuron
  # input_shape is the amount of columns from training set
  # activation function must be sigmoid, because this is a classification problem
  model = keras.Sequential([
        layers.Input(shape=[len(train_x.columns)]),
        layers.Dense(1, activation = 'sigmoid')
  ])

  # Defining the optimizer
  optimizer = tf.keras.optimizers.RMSprop(
      learning_rate = 0.001)

  # Binary Cross Entropy is the default loss function in classification models
  model.compile(loss = 'binary_crossentropy',
      optimizer = optimizer,
      metrics = ['binary_crossentropy','binary_accuracy'])

  return model

Just for curiosity, you should observe how many parameters ($\theta$) your model has.
At this point, your model is built.

In [ ]:
model = build_model()
model.summary()

After creating the model, it must be trained (fitted).
Training is done using training set and the amount of epochs must be defined.

In [ ]:
EPOCHS = 500

history = model.fit(
    train_x, train_y_bin, epochs = EPOCHS, verbose = 1
)

This plot should be generated just to inspect the learning convergence.
It is expected a decreasing of the loss function value through the epochs.


In [ ]:
plt.plot(history.history['binary_crossentropy'])
plt.title('Training Binary Cross Entropy')
plt.ylabel('Binary Cross Entropy')
plt.xlabel('Epoch')
plt.legend(['Error'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['binary_accuracy'])
plt.title('Training Binary Accuracy')
plt.ylabel('Binary Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy'], loc='lower right')
plt.show()

After the training process, the knowledge learnt by a neural network is stored in its weights.

In [ ]:
weights, biases = model.get_weights() # return a numpy list of weights
print(weights)
plt.plot(weights)
plt.ylabel('Weights')
plt.xlabel('Inputs')

In [ ]:
plt.barh(train_x.columns, weights[:,0].astype(float), align='center')
plt.xlabel("Weights")
plt.ylabel("Inputs")
#plt.title(target)
plt.savefig("NN-Weights.png")

In [ ]:
test_predictions = model.predict(test_x).flatten() # predict radon activities with the built linear regression model
test_predictions1 = test_predictions > 0.5 #np.around(test_predictions)  # > 0.5
tp = np.count_nonzero((test_predictions1 == 1) & (np.transpose(test_y_bin) == 1))
tn = np.count_nonzero((test_predictions1 == 0) & (np.transpose(test_y_bin) == 0))
accuracy_test = (tp + tn)/len(test_y)
print('The accuracy on the test set is equal to: %.4f %%' % (accuracy_test*100))

In [ ]:
# Generate classification report
print(classification_report(test_y_bin, test_predictions1, target_names=["Not Fire", "Fire"]))

In [ ]:
train_predictions = model.predict(train_x).flatten() # predict radon activities with the built linear regression model
train_predictions1 = train_predictions > 0.5
tp = np.count_nonzero((train_predictions1 == 1) & (np.transpose(train_y_bin) == 1))
tn = np.count_nonzero((train_predictions1 == 0) & (np.transpose(train_y_bin) == 0))
accuracy_train = (tp + tn)/len(train_y)
print('The accuracy on the training set is equal to: %.4f %%.' % (accuracy_train*100))

In [ ]:
# Generate classification report
print(classification_report(train_y_bin, train_predictions1, target_names=["Not Fire", "Fire"]))

After the training process, the model should be tested in order to measure its quality, it means, how good are its predictions. The model must be evaluated using the testing set, which is composed by samples that are not in the training set. In classification problems, the accuracy is the default metric.
The accuracy is computed using real outputs ($y$) and predicted outputs ($\hat{y}$). Accuracy can vary between 0 (bad predictions) and 1 (perfect predictions). Accuracy also can be presented in percentage, ranging from 0 to 100%.

In [ ]:
# Save the model architecture as an image
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)